# Introduction

Après le rendez-vous du 18/03 avec Thomas, il nous a été conseillé de se concentrer maintenant sur la temporalité dans les données et la création d'une variable target. 

On se concentrera ici sur la target  
Contrairement aux précédents notebooks (top10), l'exploration ici se fait sur l'ensemble des données : les indicateurs financiers fondamentaux de 100 entreprises au Nasdaq de 2020 à 2024



# Préparation des données et import

In [62]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go 
from plotly.subplots import make_subplots

# Import du csv précédemment utilisé
df = pd.read_csv('../csv/nasdaq100_fundamentals_fpm.csv')

print(df.symbol.unique())
df = df.drop(15) # Un index de google

['ADBE' 'AMD' 'ABNB' 'GOOGL' 'GOOG' 'AMZN' 'AEP' 'AMGN' 'ADI' 'ANSS'
 'AAPL' 'AMAT' 'APP' 'ARM' 'ASML' 'AZN' 'TEAM' 'ADSK' 'ADP' 'AXON' 'BKR'
 'BIIB' 'BKNG' 'AVGO' 'CDNS' 'CDW' 'CHTR' 'CTAS' 'CSCO' 'CCEP' 'CTSH'
 'CMCSA' 'CEG' 'CPRT' 'CSGP' 'COST' 'CRWD' 'CSX' 'DDOG' 'DXCM' 'FANG'
 'DASH' 'EA' 'EXC' 'FAST' 'FTNT' 'GEHC' 'GILD' 'GFS' 'HON' 'IDXX' 'INTC'
 'INTU' 'ISRG' 'KDP' 'KLAC' 'KHC' 'LRCX' 'LIN' 'LULU' 'MAR' 'MRVL' 'MELI'
 'META' 'MCHP' 'MU' 'MSFT' 'MSTR' 'MDLZ' 'MDB' 'MNST' 'NFLX' 'NVDA' 'NXPI'
 'ORLY' 'ODFL' 'ON' 'PCAR' 'PLTR' 'PANW' 'PAYX' 'PYPL' 'PDD' 'PEP' 'QCOM'
 'REGN' 'ROP' 'ROST' 'SBUX' 'SNPS' 'TTWO' 'TMUS' 'TSLA' 'TXN' 'TTD' 'VRSK'
 'VRTX' 'WBD' 'WDAY' 'XEL' 'ZS']


Google est en double sous deux tickers différents, avec les mêmes valeurs  
On supprime donc un des doublons

In [63]:
# Filtrer les données de 2024 pour comparaison et calculer la marge de profit
df['marginProfit'] = df.netIncomePerShare / df.revenuePerShare
nsdq24 = df[df.calendarYear == 2024].copy()

La **marge de profit** fait partie de nos indicateurs, on la crée donc ici   

Bénéfice net par action : netIncomePerShare  
Revenu par action : revenuePerShare  
Formule : **netMargin = (netIncomePerShare / revenuePerShare)**

Par rapport aux autres notebooks, on rajoute ici un indicateur **debtToEquity** afin de mesurer la solidité d'une entreprise.  
L'indicateur mesure les **fonds propres (Actifs - Dettes)**

In [64]:
# On affiche la distribution des variables
indicators = ['peRatio', 'pbRatio', 'marginProfit', 'roe', 'dividendYield', 'debtToEquity']

fig = make_subplots(rows = 2, cols = 3)
for i, indicator in enumerate(indicators):
    box = px.strip(df[indicator], hover_name=df['symbol'])
    
    # Division integer et modulo pour indiquer les rows/cols dans la boucle     
    fig.add_trace(box.data[0], row=i//3+1, col=(i%3)+1) 

fig.update_layout(title='Distribution des variables', 
                  width=1300, 
                  height=900)\
                  .show()

On constate ici que certaines valeurs extrêmes (non aberrantes car possibles) influencent grandement la distribution, et donc les **calculs de corrélation**.

On fait abstraction ici de ces valeurs extrêmes

In [65]:
df_filtered = nsdq24.loc[(nsdq24['symbol'] != 'VRSK') & (nsdq24['symbol'] != 'HON') & (nsdq24['symbol'] != 'MSTR') & (nsdq24['symbol'] != 'MDB') & (nsdq24['symbol'] != 'MELI')]
df_filtered.symbol.unique()

array(['ADBE', 'AMD', 'ABNB', 'GOOG', 'AMZN', 'AEP', 'AMGN', 'ADI',
       'ANSS', 'AAPL', 'AMAT', 'APP', 'ARM', 'ASML', 'AZN', 'TEAM',
       'ADSK', 'ADP', 'AXON', 'BKR', 'BIIB', 'BKNG', 'AVGO', 'CDNS',
       'CDW', 'CHTR', 'CTAS', 'CSCO', 'CTSH', 'CMCSA', 'CEG', 'CPRT',
       'CSGP', 'COST', 'CRWD', 'CSX', 'DDOG', 'DXCM', 'FANG', 'DASH',
       'EA', 'EXC', 'FAST', 'FTNT', 'GEHC', 'GILD', 'GFS', 'IDXX', 'INTC',
       'INTU', 'ISRG', 'KDP', 'KLAC', 'KHC', 'LRCX', 'LIN', 'MAR', 'MRVL',
       'META', 'MCHP', 'MU', 'MSFT', 'MDLZ', 'MNST', 'NFLX', 'NVDA',
       'NXPI', 'ORLY', 'ODFL', 'ON', 'PCAR', 'PLTR', 'PANW', 'PAYX',
       'PYPL', 'PEP', 'QCOM', 'REGN', 'ROP', 'ROST', 'SBUX', 'SNPS',
       'TTWO', 'TMUS', 'TSLA', 'TXN', 'TTD', 'VRTX', 'WBD', 'WDAY', 'XEL',
       'ZS'], dtype=object)

In [66]:
fig = make_subplots(rows = 2, cols = 3)
for i, indicator in enumerate(indicators):
    box = px.strip(df_filtered[indicator], hover_name=df_filtered['symbol'])
    fig.add_trace(box.data[0], row=i//3+1, col=(i%3)+1)
    
fig.update_layout(title='Distribution des variables', 
                  width=1300, height=900).show()

On remarque ici que les distributions sont plus lisibles, on procède maintenant aux calculs de corrélation

# Corrélation des variables

In [67]:
methods = ['pearson', 'spearman']
titles = ['Linéaire', 
          'Monotonique']

fig = make_subplots(rows = 1, cols = 2)
for i, (method, title) in enumerate(zip(methods, titles)):
    heatmap = px.imshow(df_filtered[indicators]\
                           .corr(method=method)\
                           .round(2), 
                           text_auto=True,
                           color_continuous_scale='Plasma',
                           title=title)
    
    fig.add_trace(heatmap.data[0], row=1, col=i+1)
    
fig.update_layout(title='Matrices de corrélation', 
                  coloraxis=dict(colorscale='Plasma'), 
                  width= 1000, 
                  annotations=[dict(text=titles[i],
                               x=0.18 + 0.65*i,
                               y=1.1,
                               xref="paper", 
                               yref="paper",
                               showarrow=False) for i in range(2) ]).show()

# Interprétations
**1. Analyse de la Rentabilité**
- **Corrélation entre ROE et Margin Profit (0.61 en monotone, 0.41 en linéaire)**
  - La rentabilité des fonds propres (*Return on Equity - ROE*) et la marge bénéficiaire (*Margin Profit*) sont naturellement liées.
  - La **forte corrélation monotone** suggère qu’une entreprise avec une bonne marge bénéficiaire a tendance à afficher un ROE élevé.
  - La **corrélation linéaire faible** indique que d’autres facteurs (structure du capital, effet de levier) influencent le ROE.
  - **Interprétation** : Une entreprise avec une marge élevée **n'a pas forcément un ROE élevé** si elle est fortement endettée ou si elle réinvestit massivement ses bénéfices.
- **Corrélation entre ROE et P/B Ratio (0.8 en linéaire, 0.41 en linéaire)**
  - La rentabilité des fonds propres (*Return on Equity - ROE*) et le  (*price to book Ratio*) sont naturellement liées puisque  la valeur comptable par action est calculée par les fonds propres.

---

**2. Effet de levier : Debt-to-Equity et ROE**
- **Corrélation entre Debt-to-Equity et ROE (0.64 en linéaire, 0.44 en monotone)**
  - La relation **positive** signifie que plus une entreprise est endettée, plus son ROE a à monter.
  - Un fort *Debt-to-Equity* signifie un financement important par la dette, ce qui **diminue les fonds propres (equity)** et donc a naturellement tendance à réduire dénominateur du ROE.
  - Certaines entreprises utilisent l’endettement pour améliorer leur ROE (effet de levier). Si la rentabilité des investissements dépasse le coût de la dette, le ROE peut augmenter.
  - **Interprétation** : Une **corrélation positive forte** indique ici que les entreprises avec un endettement élevé augmentent leur rentabiltié par rapport aux fonds propres. Cette relation est en partie mathématique mais peut aussi indiquer un effet de levier bien utilisé.

---

**3. Valorisation et Rentabilité : P/E Ratio et ROE**
- **Corrélation entre PE Ratio et ROE (0.1 en monotone, 0 en linéaire)**
  - Le Price-to-Earnings Ratio (*P/E Ratio*) mesure combien les investisseurs sont prêts à payer pour 1€ de bénéfice.
  - Une **faible corrélation monotone** signifie qu’il existe une tendance où des entreprises avec un ROE élevé ont un *P/E* plus élevé.
  - La **corrélation linéaire nulle** suggère que la relation n’est pas uniforme : certaines entreprises avec un ROE élevé peuvent être sous-évaluées (P/E faible) si le marché doute de leur croissance.
  - **Interprétation** : Un ROE élevé **ne garantit pas** un P/E élevé. Les investisseurs ne peuvent se baser uniquement sur la rentabilité.

---

**4. Valorisation et Structure Financière : PB Ratio et Debt-to-Equity**
- **Corrélation entre PB Ratio et Debt-to-Equity (0.49 en linéaire)**
  - Le Price-to-Book Ratio (*PB Ratio*) compare la valeur marchande d’une entreprise à sa valeur comptable.
  - La **corrélation positive** signifie que les entreprises fortement endettées ont un PB Ratio élevé ce qui est logique car la dette augmentant, la valeur comptable baisse (dénominateur) et l'indicateur P/B Ratio augmente.

---

**5. Impact des Dividendes : Dividend Yield et les autres ratios**
- **Faible corrélation du Dividend Yield avec les autres variables (-0.29 à 0.17)**
  - Un *Dividend Yield* (rendement du dividende) faible corrélé aux autres variables signifie que les entreprises qui versent des dividendes ne sont pas nécessairement les plus rentables, endettées ou bien valorisées.
  - **Explication** : Les entreprises à forte croissance **réinvestissent leurs bénéfices** plutôt que de verser des dividendes, ce qui explique le faible lien avec ROE et PE Ratio.
  - **Interprétation** : Un Dividend Yield élevé peut être **le signe d’une entreprise mature** avec peu de croissance future, alors qu’un faible Dividend Yield peut indiquer une **stratégie axée sur la croissance**.

---

**Application en investissement :**
- Privilégier les entreprises avec **une bonne marge bénéficiaire et un faible endettement**.
- Éviter les entreprises avec un **PB Ratio et un Debt-to-Equity élevés sans rentabilité claire**.
- Un faible P/E Ratio avec un bon ROE peut être une **opportunité sous-évaluée**.

En résumé, les P/B et P/E ratios mesurent la **confiance du marché** dans l'action.
Le ROE mesure la **rentabilité**.
Un debtToEquity faible à tendance à être une **preuve de solidité**.

Une bonne approche serait de regarder les entreprises qui maximisent le ROE tout en minimisant le D/E et le P/B Ratio ou le P/E Ratio.  
  
Comment choisir entre P/B Ratio et P/E Ratio ? 
**consommation** est plus pertinent pour les **consommation**(banques, industrie, immobilier) car basé sur la valeur comptable.
**consommation** est plus pertinent pour les entreprises générant des bénéfices constants ou avec beaucoup d'intangibles, notamment la **consommation** ou la **consommation**.
Pour le NASDAQ, on préfèrera donc le **P/E Ratio**.

Une entreprise avec un **ROE fort**(rentable par rapport à ses fonds propres), un **P/E ratio faible** (peu de confiance du marché, soit car trop risquée soit car sous évaluée) et **un D/E faible** (peu endettée donc solide) peut indiquer une opportunité boursière.

On visualise cette hypothèse dans un graphique 3D



# Opportunités boursières

In [68]:
px.scatter_3d(df_filtered, 
              x="roe", 
              y="debtToEquity", 
              z="peRatio", 
              color="symbol", 
              hover_name='calendarYear',
              title = 'Visualisation des opportunités boursières',
              size_max=10).show()

On observe ici que VRTX et ZS ont toutes deux les potentielles meilleures opportunités en 2024 selon notre hypothèse.  

# Suite du projet
Les conclusions sont intéressantes, mais le données ne semblent pas fiables en comparant les valeurs de peRatio par exemple avec d'autres sources. Il conviendrait donc de tester d'autres sources